In [18]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch, helpers
import json
from flask import Flask, request
from flask_cors import CORS

In [2]:
def import_dataset(dataset):
    dataset = pd.read_parquet(dataset)
    return dataset

In [3]:
recipes = import_dataset('resources/input/recipes.parquet')

In [4]:
recipes

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379.0,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,PT35M,PT1H,PT1H35M,2020-12-22 15:27:00+00:00,Make and share this Meg's Fresh Ginger Gingerb...,[],...,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,[Preheat oven to 350&deg;F Grease an 8x8 cake ...
522513,541380.0,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,PT3H,PT30M,PT3H30M,2020-12-22 15:32:00+00:00,"White, black, green, and pink peppercorns add ...",[https://img.sndimg.com/food/image/upload/w_55...,...,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,[Position rack in center of oven and preheat t...
522514,541381.0,Kirshwasser Ice Cream,2001131545,Jonathan F.,PT3H,PT1H,PT4H,2020-12-22 15:33:00+00:00,Make and share this Kirshwasser Ice Cream reci...,[],...,72.6,470.9,192.5,33.9,0.0,17.3,12.8,6.0,None,[heat half and half and heavy cream to a simme...
522515,541382.0,Quick & Easy Asian Cucumber Salmon Rolls,2001004241,CLUBFOODY,None,PT15M,PT15M,2020-12-22 22:11:00+00:00,"Extremely quick and easy to make, these are gr...",[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,2.9,100.5,0.3,0.0,0.1,2.4,NaN,20 rolls,"[In a small bowl, combine mayo and wasabi past..."


In [5]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   RecipeId                    522517 non-null  float64            
 1   Name                        522517 non-null  object             
 2   AuthorId                    522517 non-null  int32              
 3   AuthorName                  522517 non-null  object             
 4   CookTime                    439972 non-null  object             
 5   PrepTime                    522517 non-null  object             
 6   TotalTime                   522517 non-null  object             
 7   DatePublished               522517 non-null  datetime64[us, UTC]
 8   Description                 522512 non-null  object             
 9   Images                      522516 non-null  object             
 10  RecipeCategory              521766 non-null 

In [6]:
recipes.isna().sum()

RecipeId                           0
Name                               0
AuthorId                           0
AuthorName                         0
CookTime                       82545
PrepTime                           0
TotalTime                          0
DatePublished                      0
Description                        5
Images                             1
RecipeCategory                   751
Keywords                           0
RecipeIngredientQuantities         0
RecipeIngredientParts              0
AggregatedRating              253223
ReviewCount                   247489
Calories                           0
FatContent                         0
SaturatedFatContent                0
CholesterolContent                 0
SodiumContent                      0
CarbohydrateContent                0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                182911
RecipeYield                   348071
R

In [7]:
recipes.loc[recipes['RecipeCategory'].isna()]['RecipeCategory']


1369      None
75670     None
76570     None
80674     None
80701     None
          ... 
521877    None
521925    None
522158    None
522455    None
522486    None
Name: RecipeCategory, Length: 751, dtype: object

In [8]:
recipes[['AggregatedRating','ReviewCount']].sort_values(by='ReviewCount', ascending=False).fillna(0)

,AggregatedRating,ReviewCount
41924,5.0,3063.0
1436,5.0,2273.0
23725,5.0,1692.0
84057,4.5,1657.0
35377,5.0,1586.0
...,...,...
522512,0.0,0.0
522513,0.0,0.0
522514,0.0,0.0
522515,0.0,0.0


In [9]:
recipes.loc[recipes['RecipeServings'].isna()]['RecipeServings']


8        NaN
9        NaN
12       NaN
15       NaN
23       NaN
          ..
522505   NaN
522506   NaN
522511   NaN
522515   NaN
522516   NaN
Name: RecipeServings, Length: 182911, dtype: float64

In [10]:
recipes.iloc[522505].RecipeIngredientParts


array(['green chilies', 'cheddar cheese', 'eggs',
       'half & half light cream', 'milk', 'Wondra Flour', 'salt'],
      dtype=object)

In [11]:
for row in recipes.loc[recipes['Images'].isna()].index:
    print(recipes.loc[row, 'Images'])

None


In [12]:
cleaned_recipes = recipes
cleaned_recipes['CookTime'] = cleaned_recipes['CookTime'].fillna('')
cleaned_recipes['AggregatedRating'] = cleaned_recipes['AggregatedRating'].fillna(0)
cleaned_recipes['ReviewCount'] = cleaned_recipes['ReviewCount'].fillna(0)
cleaned_recipes['Description'] = cleaned_recipes['Description'].fillna('')
cleaned_recipes['RecipeCategory'] = cleaned_recipes['RecipeCategory'].fillna('')
cleaned_recipes['RecipeServings'] = cleaned_recipes['RecipeServings'].fillna(0)
cleaned_recipes['RecipeYield'] = cleaned_recipes['RecipeYield'].fillna('0')
for row in cleaned_recipes.loc[cleaned_recipes['Images'].isna()].index:
    cleaned_recipes.at[row, 'Images'] = []
cleaned_recipes.isna().sum()

RecipeId                      0
Name                          0
AuthorId                      0
AuthorName                    0
CookTime                      0
PrepTime                      0
TotalTime                     0
DatePublished                 0
Description                   0
Images                        0
RecipeCategory                0
Keywords                      0
RecipeIngredientQuantities    0
RecipeIngredientParts         0
AggregatedRating              0
ReviewCount                   0
Calories                      0
FatContent                    0
SaturatedFatContent           0
CholesterolContent            0
SodiumContent                 0
CarbohydrateContent           0
FiberContent                  0
SugarContent                  0
ProteinContent                0
RecipeServings                0
RecipeYield                   0
RecipeInstructions            0
dtype: int64

In [13]:
cleaned_recipes.isna().sum()

RecipeId                      0
Name                          0
AuthorId                      0
AuthorName                    0
CookTime                      0
PrepTime                      0
TotalTime                     0
DatePublished                 0
Description                   0
Images                        0
RecipeCategory                0
Keywords                      0
RecipeIngredientQuantities    0
RecipeIngredientParts         0
AggregatedRating              0
ReviewCount                   0
Calories                      0
FatContent                    0
SaturatedFatContent           0
CholesterolContent            0
SodiumContent                 0
CarbohydrateContent           0
FiberContent                  0
SugarContent                  0
ProteinContent                0
RecipeServings                0
RecipeYield                   0
RecipeInstructions            0
dtype: int64

In [14]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   RecipeId                    522517 non-null  float64            
 1   Name                        522517 non-null  object             
 2   AuthorId                    522517 non-null  int32              
 3   AuthorName                  522517 non-null  object             
 4   CookTime                    522517 non-null  object             
 5   PrepTime                    522517 non-null  object             
 6   TotalTime                   522517 non-null  object             
 7   DatePublished               522517 non-null  datetime64[us, UTC]
 8   Description                 522517 non-null  object             
 9   Images                      522517 non-null  object             
 10  RecipeCategory              522517 non-null 

In [15]:
cleaned_recipes.to_parquet('resources/output/cleaned_recipes.parquet.gzip', compression='gzip')

In [16]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import json

class Indexer:
    def __init__(self):
        self.df = pd.read_parquet('resources/output/cleaned_recipes.parquet.gzip')
        self.es_client = Elasticsearch("https://localhost:9200", basic_auth=("elastic","gKzjmdkd=QKeITShRujT"), ca_certs="resources/input/private/http_ca.crt")

    def run_indexer(self):
        self.es_client.indices.create(index='spatial-food', ignore=400)
        self.es_client.indices.delete(index='spatial-food', ignore=[400, 404])
        self.df['_index'] = 'spatial-food'
        j = json.loads(self.df.to_json(orient='records'))
        helpers.bulk(self.es_client, j)

indexer = Indexer()
indexer.run_indexer()


/var/folders/j7/hhs6jm2j6mv0cdm7n2z33sx80000gp/T/ipykernel_75735/3253146044.py:11: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  self.es_client.indices.create(index='spatial-food', ignore=400)
/var/folders/j7/hhs6jm2j6mv0cdm7n2z33sx80000gp/T/ipykernel_75735/3253146044.py:12: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  self.es_client.indices.delete(index='spatial-food', ignore=[400, 404])


In [17]:
def search(query):
    results = indexer.es_client.search(index='spatial-food', query={
            "combined_fields": {
                "query": query,
                "fields": ["Name","RecipeIngredientParts","RecipeInstructions"]
            }
        }, suggest_field='Name', suggest_text=query, suggest_mode='missing')
    print("Got %d Hits:" % results['hits']['total']['value'])
    for hit in results['hits']['hits']:
        print("The recipe is '{0}' '{1}' '{2}'.".format(hit['_source']["RecipeId"],hit['_source']["Name"],hit['_score']))
    for op in results['suggest']['Name']:
        print(op['options'])

In [25]:
app = Flask(__name__)
CORS(app)

app.es_client = Elasticsearch("https://localhost:9200", basic_auth=("elastic","gKzjmdkd=QKeITShRujT"), ca_certs="resources/input/private/http_ca.crt")
app.df = pd.read_parquet('resources/output/cleaned_recipes.parquet.gzip')

@app.route('/recipe', methods=['GET'])
def get_recipe():
    response_object = {'status':200}
    rid = int(request.args.get('rid'))
    recipe = json.loads(app.df.loc[app.df['RecipeId'] == rid ].iloc[0].to_json())
    response_object['recipe'] = recipe
    return response_object

@app.route('/recipes', methods=['GET'])
def get_recipes():
    response_object = {'status':200}
    page_size = 50  # Number of recipes per page
    page = int(request.args.get('page', 1))  # Get the page number from the request, default to 1 if not provided
    start = (page - 1) * page_size  # Calculate the start index
    end = start + page_size  # Calculate the end index
    recipe_list = json.loads(app.df[start:end].to_json(orient='records'))  # Get the recipes for this page
    response_object = recipe_list
    return response_object

@app.route('/search', methods=['GET'])
def search_recipes():
    response_object = {'status':200}
    query_term = request.args.get('query')
    results = app.es_client.search(index='foodir', query={
            "script_score":{
                "query":{
                    "dis_max": {
                        "queries": [
                            { "match": { "Name": query_term }},
                            { "match": { "RecipeIngredientParts": query_term }},
                            { "match": { "RecipeInstructions": query_term }},
                        ],
                        "tie_breaker": 0.3
                    },
                },
                "script":{
                    "source": "_score * (((doc['ReviewCount'].value * doc['AggregatedRating'].value) + (3 * 32)) / (doc['ReviewCount'].value + 32 ))"
                }
            }
        }, suggest_field='Name', suggest_text=query_term, suggest_mode='missing', size=12)
    
    total_hit = results['hits']['total']['value']
    results_df = pd.DataFrame([[hit['_source'][key] for key in hit['_source']] for hit in results['hits']['hits']], columns=list(app.df.columns))
    results_df['_score'] = [hit['_score'] for hit in results['hits']['hits']]
    response_object['total_hit'] = total_hit
    response_object['results'] = results_df.to_dict('records')
    response_object['suggest'] = results['suggest']
    return response_object

if __name__ == '__main__':
    app.run(debug=False, port=9090) 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9090
Press CTRL+C to quit
127.0.0.1 - - [18/Mar/2024 19:05:44] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:06:02] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:07:12] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:07:19] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:07:33] "GET /search?query=mushroom&page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:07:36] "GET /search?query=mushroon&page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:07:38] "GET /search?query=mushrooms&page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:07:41] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:09:49] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:10:36] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:11:09] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2024 19:22:43] "GET /recipes?page=1 HTTP/1.1" 200 -
127.0.0.1 - 